### Update job status in job-execution detail table

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("env", "", "Env Name")
dbutils.widgets.text("proj_id", "", "Project ID")
dbutils.widgets.text("dept_id", "", "Department ID")
dbutils.widgets.text("job_id", "", "Job ID")
dbutils.widgets.text("status", "", "Status")
dbutils.widgets.text("error_message", "", "Error Message")
dbutils.widgets.text("workflow_id", "", "Workflow ID")
dbutils.widgets.text("run_id", "", "Run ID")
# dbutils.widgets.text("run_url", "", "Run URL")

In [0]:
env = dbutils.widgets.get("env")
proj_id = dbutils.widgets.get("proj_id")
dept_id = dbutils.widgets.get("dept_id")
job_id = dbutils.widgets.get("job_id")
status = dbutils.widgets.get("status")
error_message = dbutils.widgets.get("error_message")
workflow_id = dbutils.widgets.get("workflow_id")
run_id = dbutils.widgets.get("run_id")

In [0]:
batch_id = dbutils.jobs.taskValues.get(taskKey="batch_check", key="batch_id", debugValue=1)

In [0]:
result = spark.sql(f"""
    select parent_job_id from data_migration_validator_dev.test_megha.job_parameter_detail 
    where proj_id = '{proj_id}' and dept_id = '{dept_id}' and job_id = '{job_id}'"""
).collect()

if result:
    parent_job_id = result[0]['parent_job_id']
else:
    parent_job_id = None

In [0]:
print(f"env: {dbutils.widgets.get('env')}")
print(f"proj_id: {dbutils.widgets.get('proj_id')}")
print(f"dept_id: {dbutils.widgets.get('dept_id')}")
print(f"job_id: {dbutils.widgets.get('job_id')}")
print(f"parent_job_id: {parent_job_id}")
print(f"batch_id: {batch_id}")
print(f"status: {dbutils.widgets.get('status')}")
print(f"error_message: {dbutils.widgets.get('error_message')}")
print(f"workflow_id: {dbutils.widgets.get('workflow_id')}")
print(f"run_id: {dbutils.widgets.get('run_id')}")

In [0]:
from src.table_utils import log_to_job_execution_detail

In [0]:
try:
    result = log_to_job_execution_detail(
        spark,
        batch_id,
        proj_id, 
        dept_id, 
        job_id,
        parent_job_id, 
        status, 
        error_message,
        workflow_id,
        run_id
        )
    
    if result:
        print(f"Successfully updated job {job_id} of batch_id {batch_id} to status {status}")
    else:
        error_msg = f"Failed to update job status"
        raise Exception(error_msg)
except Exception as e:
    error_msg = f"Error updating status: {str(e)}"
    raise Exception(error_msg)
